In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re 
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from datetime import datetime
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from textblob import TextBlob
from nltk.util import ngrams
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

data = pd.read_csv('comments_500k_201901.csv')

In [3]:
data.groupby('subreddit').agg('count').sort_values('body', ascending=False)

,parent_id,subreddit_id,created_utc,author,score,body
subreddit,,,,,,
AskReddit,25639,25639,25639,25639,25639,25639
politics,10459,10459,10459,10459,10459,10459
nfl,9231,9231,9231,9231,9231,9231
funny,5206,5206,5206,5206,5206,5206
FortNiteBR,4311,4311,4311,4311,4311,4311
nba,4199,4199,4199,4199,4199,4199
The_Donald,3784,3784,3784,3784,3784,3784
dankmemes,3629,3629,3629,3629,3629,3629
memes,3534,3534,3534,3534,3534,3534


### score analysis 

In [5]:
pd.DataFrame(data.groupby('subreddit')['score'].agg('mean').sort_values().head(20))

,score
subreddit,
321,-39.000000
AnimatedStarWarsMemes,-26.000000
paradoxball,-22.000000
Jade_Chynoweth,-20.000000
NoShitSherlock,-16.666667
chipchan,-15.500000
WhyEvenWearAnything,-15.333333
PornMemes,-15.000000
MissAlice_18,-15.000000


In [6]:
pd.DataFrame(data.groupby('subreddit')['score'].agg('mean').sort_values(ascending=False).head(20))

,score
subreddit,
badtattoos,129.000000
Taxpayers,118.000000
bigboye,110.666667
ForwardsFromKlandma,108.625000
GTAGE,103.200000
GirlsWithHugePussies,99.000000
nocontextbooks,97.000000
ThatsHowThingsWork,96.000000
bird,95.200000


In [10]:
# VADER analysis

In [16]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
#     print("{}".format(str(score)))
    return score
top_20 = data.sort_values('subreddit').reset_index()
top_20['senti']= data['body'].map(lambda t: sentiment_analyzer_scores(str(t))) 
top_20['senti_neu'] = top_20.senti.apply(lambda x: x.get('neu'))
top_20['senti_pos'] = top_20.senti.apply(lambda x: x.get('pos'))
top_20['senti_neg'] = top_20.senti.apply(lambda x: x.get('neg'))

In [22]:
pd.DataFrame(top_20.groupby('subreddit')['senti_pos'].agg('mean').sort_values(ascending=False).head(20))

,senti_pos
subreddit,
RTGameCrowd,1.0
BrigitteUndone,1.0
im14andthisisfunny,1.0
Usagimimi,1.0
BDSMUK,1.0
Citra,1.0
cf4cf,1.0
MHOCHolyrood,1.0
intelnuc,1.0


In [24]:
pd.DataFrame(top_20.groupby('subreddit')['senti_neg'].agg('mean').sort_values(ascending=False).head(50))


,senti_neg
subreddit,
UnexpectedJacksfilms,1.0000
ShitConspiracistsSay,1.0000
GreekMythology,1.0000
FreeTextBook,1.0000
GoTRPcommunity,1.0000
mildlydisturbing,1.0000
Vape_Porn,1.0000
thathappend,1.0000
collingwoodfc,1.0000


In [25]:
pd.DataFrame(top_20.groupby('subreddit')['senti_neu'].agg('mean').sort_values(ascending=False).head(20))

,senti_neu
subreddit,
0w0,1.0
LowT,1.0
LucyHale,1.0
LuckyDayApp,1.0
LuckyCameraman,1.0
LucianMains,1.0
Lubuntu,1.0
LoyolaWilmete,1.0
LoveToWatchYouLeave,1.0


In [26]:
data.groupby('subreddit').agg('count').sort_values('body', ascending=False)

,parent_id,subreddit_id,created_utc,author,score,body
subreddit,,,,,,
AskReddit,25639,25639,25639,25639,25639,25639
politics,10459,10459,10459,10459,10459,10459
nfl,9231,9231,9231,9231,9231,9231
funny,5206,5206,5206,5206,5206,5206
FortNiteBR,4311,4311,4311,4311,4311,4311
nba,4199,4199,4199,4199,4199,4199
The_Donald,3784,3784,3784,3784,3784,3784
dankmemes,3629,3629,3629,3629,3629,3629
memes,3534,3534,3534,3534,3534,3534
